Naszym zadaniem jest przeanalizowanie zbioru danych medycznych oraz stworzenia modelu klasyfikującego potencjalnego pacjenta ze względu na fakt wystąpienia udaru mózgu.

Udar mózgu to zespół objawów klinicznych związanych z nagłym wystąpieniem ogniskowego lub uogólnionego zaburzenia czynności mózgu, powstały w wyniku zaburzenia krążenia mózgowego i utrzymującego się ponad 24 godziny.

# Biblioteki

In [38]:
import pandas as pd
import numpy as np

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

# Funkcje

In [128]:
def plotting(data: pd.DataFrame, x: str, type: str, cl = 'stroke'):
  """
    Function for plotting data in specific way

  Args:
    data - dataset to plot

    x - column for x-axis
    
    type - type of plot 
      values: {'hist', 'box', 'hist_box', 'class_hist', 'class_box', 'class_hist_box'} 

  Returns:
    Plot in plotly
  """

  def plot_class_hist():
    """
      Plots histogram with stroke classes in plotly
    """
    
    fig = go.Figure()
    fig.add_trace(go.Histogram(x = data.loc[data[cl]==0, x], name='No '+cl))
    fig.add_trace(go.Histogram(x = data.loc[data[cl]==1, x], name=cl))

    fig.update_layout(barmode='overlay')
    fig.update_traces(opacity=0.75)
    
    return fig
  
  def plot_class_box():
    """
      Plots boxplot with stroke classes in plotly
    """
    
    fig = go.Figure()
    fig.add_trace(go.Box(y = data.loc[data[cl]==0, x], name='No '+cl))
    fig.add_trace(go.Box(y = data.loc[data[cl]==1, x], name=cl))

    fig.update_layout(barmode='overlay')
    fig.update_traces(opacity=0.75)
    
    return fig
    

  if type=='hist':
    fig = px.histogram(data, x=x)
    fig.show()

  elif type=='box':
    fig = px.box(data, x=x)
    fig.show()

  elif type=='hist_box':
    fig = make_subplots(rows=1, cols=2)

    fig.add_trace(go.Histogram(x = data[x], name=x), row=1,col=1)
    fig.add_trace(go.Box(y = data[x], name=x), row=1, col=2)
    
    fig.update_layout(barmode="overlay")
    fig.update_traces(opacity=0.7, row=1, col=1)
    fig.update_layout(title_text="Histogram and Boxplot for " + x)
    fig.show()

  elif type=='class_hist':
    
    fig = plot_class_hist()
    fig.show()
  
  elif type=='class_box':
    
    fig = plot_class_box()
    fig.show()

  elif type=='class_hist_box':
    fig = make_subplots(rows=1, cols=2)

    fig.add_trace(go.Histogram(x = data.loc[data[cl]==0, x], name='No '+cl), row=1,col=1)
    fig.add_trace(go.Histogram(x = data.loc[data[cl]==1, x], name=cl), row=1, col=1)
    fig.add_trace(go.Box(y = data.loc[data[cl]==0, x], name='No '+cl), row=1, col=2)
    fig.add_trace(go.Box(y = data.loc[data[cl]==1, x], name=cl), row=1, col=2)

    fig.update_layout(barmode="overlay")
    fig.update_traces(opacity=0.7, row=1, col=1)
    fig.update_layout(title_text="Histogram and Boxplot with classes for " + x)
    fig.show()

  else:
    raise NotImplementedError('Not implemented!')

  return 0

In [121]:
def iqr_outliers(df: pd.DataFrame, col: str):
  Q1 = df[col].quantile(0.25)
  Q3 = df[col].quantile(0.75)
  IQR = Q3 - Q1
  df.loc[df[col] <= Q1 - 1.5*IQR, col] = df[col].quantile(0.1)
  df.loc[df[col] >= Q3 + 1.5*IQR, col] = df[col].quantile(0.9)

  return df

# Dane

In [5]:
url = 'https://github.com/maju116/Statistics_II_GUT/raw/main/PROJECT/healthcare-dataset-stroke-data.csv'
df = pd.read_csv(url)

df.head(5)

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   object 
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   object 
 6   work_type          5110 non-null   object 
 7   Residence_type     5110 non-null   object 
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     5110 non-null   object 
 11  stroke             5110 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 479.2+ KB


In [5]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
id,5110.0,36517.829354,21161.721625,67.00,17741.250,36932.000,54682.00,72940.00
age,5110.0,43.226614,22.612647,0.08,25.000,45.000,61.00,82.00
hypertension,5110.0,0.097456,0.296607,0.00,0.000,0.000,0.00,1.00
heart_disease,5110.0,0.054012,0.226063,0.00,0.000,0.000,0.00,1.00
avg_glucose_level,5110.0,106.147677,45.283560,55.12,77.245,91.885,114.09,271.74
bmi,4909.0,28.893237,7.854067,10.30,23.500,28.100,33.10,97.60
stroke,5110.0,0.048728,0.215320,0.00,0.000,0.000,0.00,1.00


Mamy łącznie 12 kolumn po 5110 obserwacji z czego jedna z nich stanowi ('stroke') zmienną celu i będziemy starać się ja zamodelować. Jest to zmienna binarna informująca czy dany pacjent miał udar mózgu czy też nie zatem jest to typowe zadanie klasyfikujące. Oprócz tego jest pozostałych 11 zmiennych:
- zmienna **id** typu integer przypisująca konkretnego pacjenta nie wnosi raczej żadnej wartości informacyjnej ale pozwoli nam sprawdzić czy jakiś pacjent jest dwukrotnie w historii co mogłoby wskazywać na podwyższone ryzyko udaru.
- zmienna **gender** typu object jest zmienną binarną wskazująca na płeć pacjenta, z pewnością będzie trzeba ją zakodować. Spodziewa się, iż będzie dawała istotną informacje dla zmiennej celu. 
- zmienna **age** typu float jest zmienną numeryczną wskazująca na wiek pacjenta i także można się spodziewać że będzie istotna w modelowaniu. Mamy tutaj zakres wartości od 25 do 82 bez raczej mocnych ogonów.
- zmienna **hypertension** typu integer jest zmienną binarną informująca o nadciśnieniu pacjenta.
- zmienna **heart_disease** typu integer jest zmienną binarną informująca o występowaniu chorób serca.
- zmienna **ever_married** typu object jest zmienną kategoryczną informująca o tym czy pacjent kiedykolwiek był w związku małżeńskim.
- zmienna **work_type** typu object jest zmienną kategoryczną informująca o typie zatrudnieniu.
- zmienna **residence_type** typu object jest zmienną kategoryczną informująca o strukturze miejsca zamieszkania.
- zmienna **avg_glucose_level** typu float jest zmienną numeryczną informująca o przeciętnyn poziomie glukozy we krwi.
- zmienna **bmi** typu float jest zmienną numeryczną informująca o wskaźniku BMI pacjenta. Posiada 201 braków danych.
- zmienna **smoking_status** typu object jest zmienną kategoryczną informująca o tym czy i jak często palił pacjent.
- zmienna **stroke** typu int jest zmienną binarną informująca czy pacjent doznał udaru mózgu i jest to nasza zmienną celu.

Widzimy, że mamy 211 braków danych dla zmiennej **bmi**, którymi zajmiemy się w dalszej części projektu.

# EDA

### stroke

In [6]:
df.stroke.value_counts()

0    4861
1     249
Name: stroke, dtype: int64

In [ ]:
plotting(df, 'stroke', type = 'hist')

0

Widzimy, że nasza zmienna celu jest niezbalansowana gdzie klasą większościową są pacjenci bez udaru mózgu. Musimy mieć to na uwadze i w przypadku pozostawienia takiego stanu rzeczy nie możemy skorzystać z metryki **accuracy**, a powinniśmy skorzystać z innej - przykładowo średniej harmonicznej między **precision** a **recall** czyli **f1-score**. Niezbalansowany zbiór danych jest jednak dość problemtyczny, gdyż klasyfikator ma tendencję skupiania się na predykcji klasy większościowej. 

Istnieją sposoby na zbalansowanie zbioru danych, które są częścią **pre-processingu** czyli wstępnego przetwarzania danych przed modelowaniem. Do jednych z bardziej popularnych metod należą:
- **undersampling** - pozostawia wszystkie obserwacje z klasy mniejszościowej i losowo eliminuje obiekty z klasy większościowej
- **oversampling** - pozostawia wszystkie obserwacje z klasy większościowej i losowo replikuje elementy z klasy mniejszościowej
- **SMOTE Algorithm** - wykorzystuje podejście algorytmiczne KNN tj. dla każdej obserwacji **$x$** wybiera $k$ najbliższych sąsiadów i wybiera losowo jednego **$x'$** z nich. Następnie różnica między koordynatami **$x$** oraz **$x'$** jest obliczana i przemnożona losowo przez liczby z przedziału $(0,1)$. Tak stworzona z różnic obserwacja **$x''$** jest dodana do zbioru. Geometrycznie odwzorowuje to stworzenie nowej obserwacji na podstawie przesunięcia obserwacji bieżącej do jednego z sąsiadów.

W przypadku undersamplingu możemy pozbyć się istotnych informacyjnie obserwacji. Dodatkowo nasz zbiór danych jest niewielki, więc nie chcielibyśmy się ograniczyć do zaledwie 249 obserwacji. Natomiast oversampling może doprowadzić do nadmiernego dopasowania modelu do naszych danych. Istnieje też podejście mieszane wykorzystujące zarówno undersampling, jak też oversampling ale ciężko jest ustalić optymalną proporcję między podejściami. Spróbujemy sprawdzić empirycznie najlepszą metodę i się do niej ograniczyć. 

### id

In [ ]:
print("Liczba zduplikowanych wartości ID pacjentów: ", len(df[df.id.duplicated()]))

Liczba zduplikowanych wartości ID pacjentów:  0


W naszym zbiorze danych każdy pacjent jest wpisany unikalnie, tak więc zmienna ID raczej do niczego więcej nam się nie przyda i będziemy mogli ją pozostawić w dalszych etapach. 

### gender

In [ ]:
plotting(data = df, x='gender',  type='class_hist')

0

In [7]:
df.gender.value_counts()

Female    2994
Male      2115
Other        1
Name: gender, dtype: int64

W zmiennej gender występują 3 wartości (male, female, other). Wartość 'Other' występuje tylko jeden raz, więc można go potraktować jako brak danych. W zależności od wartości pozostałych zmiennej dla tej obserwacji albo zastąpimy ją wartością male lub female (jako tą która najczęściej występuje) albo usuniemy.

### age

In [22]:
plotting(data = df, x='age',  type='hist_box')

0

In [41]:
plotting(data = df, x='age',  type='class_hist_box')

0

Możemy zaobserwować, że udar mózgu w naszym zbiorze mają głównie osoby w podeszłym wieku (60-80 lat). Natomiast osoby bez udaru rozkładają się dość równomiernie, mediana wynosi 43 lata, gdzie dla osób z udarem 71 lat. Dodatkowo występują zmienne odstające w grupie ludzi z udarem. Sam rozkład zmiennej zbliżony jest do rozkładu jednostajnego. 

### hypertension

In [25]:
plotting(data = df, x='hypertension',  type='class_hist')

0

Można zauważyć, że znaczna większość pacjentów nie ma nadciśnienia. Co więcej, mimo braku zbalansowanego zbioru danych, więcej pacjentów z udarem mózgu nie miało przy tym nadciśnienia. 

### heart_disease

In [26]:
plotting(data = df, x='heart_disaease',  type='class_hist')

0

Podobna sytuacja jak dla zmiennej hypertension. Znaczna większość pacjentow nie ma chorób serca oraz mimo braku zbalansowania, większość osób z udarem także nie ma chorób serca. 

### ever_married

In [32]:
plotting(data = df, x='ever_married',  type='class_hist')

0

Zmienna ever_married jest zmienną kategoryczną, którą będzie trzeba zakodować w wartości (0,1). Większa część pacjentów była kiedykolwiek żonata oraz to w tej grupie udar mózgu występuje w większej ilości. 

### work_type

In [43]:
plotting(data = df, x='work_type',  type='class_hist')

0

Zmienna work_type określająca rodzaj wykonywanej pracy jest także zmienną kategoryczną, którą zakodujemy w wartości numeryczne. Najwięcej pacjentów jest z kategorii 'private', następnie równiemiernie ilościowo rozłożeni między 'children', 'self_employed', 'govt_job'. Znikoma ilość nigdy nie pracowała. Największa ilość udarów znajduje się wśród sektora prywatnego oraz samo-zatrunionego. Niewielka część w sektorze publicznym. Pojedyncze wartości występują też w grupie dzieci, ale z poprzednich wniosków najprawdopodobniej są to obserwacje odstające. Można także spróbować zkoszykować tą zmienną i rozdzielić na sektor obciążony i nieobciążony względem ryzyka udaru mózgu. 

### residence_type

In [48]:
plotting(data = df, x='Residence_type',  type='class_hist')

0

Zmienna określa obszar zamieszkania z rozdziałem na wiejski i miejski. Widać, że podział jest równomierny oraz w każdym z podziałów jest zbliżona proporcja osób po udarze do osób bez udaru mózgu, więc zmienna ta niesie ze sobą najpewniej niewielką ilość informacji dla potencjalnego modelu. 

### avg_glucose_level

In [14]:
plotting(data = df, x='avg_glucose_level',  type='hist_box')

0

In [7]:
plotting(data = df, x='avg_glucose_level',  type='class_hist_box')

0

Możemy zaobserwować, że najwięcej jest pacjentów ze średnim poziomem glukozy w przedziale 77.24 - I kwantyl, 114.09 - II kwantyl, a więc z poziomem prawidłowym lub stanem przedcukrzycowym. Wniosek ten jednak jest uogólniony, gdyż zakres ten jest nieco inny w zależności od pory czy też sposobu wykonywanego badania. Osoby z wyższym średnim poziomem glukozy we krwi są outlierami dla grupy bez udaru mózgu, natomiast dla pozostałej części osób mieszczą się one w IV kwantylu. Więc usunięcie tych obserwacji, lub też zmiana ich wartości spowodowałaby utratę być może znaczącej informacji. Zamiast tego spróbujemy wykorzystać tą zależność tworząc nową zmienną kategoryczną informującą o średnim zakresie glukozy (poziom prawidłowy, stan przedcukrzycowy, cukrzyca). 


### bmi

In [106]:
plotting(data = df, x='bmi',  type='hist_box')

0

In [16]:
plotting(data = df, x='bmi',  type='class_hist_box')

0

Rozkład zmiennej bmi przypomina rozkład normalny z cięższym prawym ogonem. Występują też outliery dla bmi > 47.5. Większość pacjentów zawiera się w przedziale 23.5 - 33.1 czyli jest z wagą prawidłową lub nadwagą I stopnia. O ile sama zmienna może nie nieść zbyt dużej informacji z uwagi, że rozkład wartości jest zbliżony dla osób z udarem, jak i bez udaru to niemniej możliwe są pewne istotne interakcje między pozostałymi zmiennymi jak chociażby średnim poziomem glukozy we krwi. W kontekście danych odstających również można stworzyć zmienną kategoryczną zgodnie z kategoriami wskaźników BMI.

Dodatkowo zmienna BMI zawiera braki danych, którymi należy się zająć. Istnieje na to wiele metod m.in. 
- usunięcie obserwacji
- wypełnienie ich statystyką opisową typu średnia, mediana
- wykorzystanie algorytmów ML takich jak KNN

Z racji, iż braków danych jest niewiele (zaledwie 4%) to wypełnimi ich statystyką opisową jako działaniem prostym, ale i skutecznym. Jak wspomniano, rozkład zmiennej jest zbliżony do normalnego więc nie ma większego znaczenia wybór między medianą, a średnią ale zważywszy na lekki ogon prawostronny wybrana zostanie mediana. Dla obserwacji odstających posłużymy się metodą IQR.

### smoking_status

In [17]:
plotting(data = df, x='smoking_status',  type='class_hist_box')

0

Ciężko wyciągnać tu szersze wnioski zważywszy na dość liczną grupę 'Unknown', Najliczniejsza grupa pacjentów nigdy nie paliła papierosów zakładając, że w grupie 'Unknown' wartości rozkładałyby się w miare równomiernie. Widać rozróżnienei dla osób z udarem, że w większości są to osoby niepalące lub palące sporadycznie. 

# Inżynieria cech

- **Przedział międzyćwiartykowy (IQR -Interquartile Range)** - różnica między trzecim a pierwszym kwartylem. Ponieważ pomiędzy tymi kwartylami znajduje się z definicji 50% wszystkich obserwacji (położonych centralnie w rozkładzie), dlatego im większa szerokość rozstępu ćwiartkowego, tym większe zróżnicowanie cechy.

In [138]:
#głęboka kopia danych

df_clear = df.copy()

### bmi

- imputacja braków danych przy użyciu mediany
- stworzenie nowej zmiennej kategorycznej na podstawie istniejącej zmiennej numerycznej
- dodanie zmiennej numerycznej bez danych odstających poprzez wykrycie ich metodą IQR oraz ograniczenie ich wartości na 0.9 kwantyl oraz 0.1 kwantyl
- log-transformacja zmiennej oraz wykrycie ich metodą IQR oraz ograniczenie ich wartości na 0.9 kwantyl oraz 0.1 kwantyl

In [139]:
#imputacja mediany w miejsca NA
df_clear['bmi_no_nan'] = df_clear.bmi.fillna(df_clear.bmi.median())

#nowa zmienna kategoryczna zgodnie ze wskaźnikami BMI
df_clear.loc[(df_clear.bmi_no_nan < 18.5), 'bmi_categorical'] = 'underweight'
df_clear.loc[(df_clear.bmi_no_nan >= 18.5) & (df_clear.bmi_no_nan <= 24.9), 'bmi_categorical'] = 'normal'
df_clear.loc[(df_clear.bmi_no_nan > 24.9) & (df_clear.bmi_no_nan <= 29.9), 'bmi_categorical'] = 'overweight'
df_clear.loc[(df_clear.bmi_no_nan > 29.9) & (df_clear.bmi_no_nan < 34.9), 'bmi_categorical'] = 'obese'
df_clear.loc[(df_clear.bmi_no_nan > 34.9), 'bmi_categorical'] = 'extremely obsese'

#zmienna numeryczna bez outlierów
df_clear['bmi_no_outliers'] = df_clear.bmi_no_nan
df_clear = iqr_outliers(df_clear, 'bmi_no_outliers')

#log-transformacja bez outlierów
df_clear['log_bmi_no_outliers'] = np.log(df_clear.bmi_no_nan)
df_clear = iqr_outliers(df_clear, 'log_bmi_no_outliers')

In [140]:
for col, types in zip(['bmi_no_nan', 'bmi_categorical', 'bmi_no_outliers', 'log_bmi_no_outliers'], ['hist_box','class_hist','hist_box','hist_box']):
  plotting(data = df_clear, x=col, type=types)

Możemy zaobserwować, że wypełnienie braków danych spowodowało peak w punkcie mediany co jest naturalnym efektem zastosowanej metody. Dodatkowo, najbardziej liczna jest grupa osób z nadwagą i to tam również najliczniej występują osoby z udarem mózgu. W przypadku usunięcia outlierów mozemy zaobserwować pozbycie się ogona prawostronnego, natomiast zastosowanie log-transformacji wydaje się nie mieć znacznego wpływu poza przeskalowaniem co będzie zbędne z racji iż w późniejszym etapie będziemy dokonywać standaryzacji zmiennych. 

# Literatura
- http://cejsh.icm.edu.pl/cejsh/element/bwmeta1.element.desklight-002cb3e1-70f4-4321-a489-f4ced00e9d3b
